In [ ]:
%%time
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold
from sklearn.metrics import *
import lightgbm as lgb
from lightgbm import LGBMRegressor

FOLDS = 10
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)

oof_lgb = np.zeros(len(train))
pred_lgb = np.zeros(len(test))

for i, (tr_idx, va_idx) in enumerate(kf.split(train)):
    
    X_train = train.loc[tr_idx, features].copy()
    y_train = train.loc[tr_idx, target]
    X_valid = train.loc[va_idx, features].copy()
    y_valid = train.loc[va_idx, target]
    X_test = test[features].copy()

    model_lgb = LGBMRegressor(
        device="gpu",
        max_depth=3,
        colsample_bytree=0.4,  
        cat_features=cat,
        #subsample=0.9, 
        n_estimators=5000, 
        learning_rate=0.02, 
        objective="regression", 
        verbose=-1, 
        early_stopping_rounds=20,
    )
    
    model_lgb.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)]
    )

    valid_preds = model_lgb.predict(X_valid)
    oof_lgb[va_idx] = valid_preds
    pred_lgb += model_lgb.predict(X_test)
    fold_rmse = mean_squared_error(y_valid, valid_preds, squared=False)
    
    print("#"*25)
    print(f"### Fold {i+1} \n")
    print(f"Fold {i+1} RMSE: {fold_rmse}")
    print("#"*25)

pred_lgb /= FOLDS